In [17]:
# Clean up any existing database and cleaned files prior to creating the DB and cleaned CSV files

In [18]:
!rm data/flp2Data.sqlite data/home_cleaned.csv data/income_cleaned.csv

rm: data/flp2Data.sqlite: No such file or directory
rm: data/home_cleaned.csv: No such file or directory
rm: data/income_cleaned.csv: No such file or directory


In [19]:
# Dependencies
import pandas as pd
import numpy as np
import os

# Data Cleaning

In [20]:
# Path of the CSV file
homecsv = "data/median_home_prices.csv"
incomecsv = "data/median_household_income.csv"

In [21]:
# Read CSV file into a pandas DataFrame
income_df = pd.read_csv(incomecsv, dtype=object)


In [22]:
# Read CSV file into a pandas DataFrame
home_df = pd.read_csv(homecsv, dtype=object)


In [23]:
# Preview DataFrame
# Note that some rows are missing gender
income_df.head()

,state,abbr,mhi_09,mhi_10,mhi_11,mhi_12,mhi_13,mhi_14,mhi_15,mhi_16
0,Alabama,AL,41216,42081,42934,43160,43253,43511,43623,44758
1,Alaska,AK,64635,66521,69014,69917,70760,71829,72515,74444
2,Arizona,AZ,50296,50448,50752,50256,49774,49928,50255,51340
3,Arkansas,AR,38542,39267,40149,40531,40768,41264,41371,42336
4,California,CA,60392,60883,61632,61400,61094,61489,61818,63783


In [24]:
home_df.head()

,state,abbr,mhp_09,mhp_10,mhp_11,mhp_12,mhp_13,mhp_14,mhp_15,mhp_16
0,Alabama,AL,111900,117600,120800,122300,122500,123800,125500,128500
1,Alaska,AK,221300,229100,235100,237900,241800,246300,250000,257100
2,Arizona,AZ,218400,215000,197400,175900,165100,162900,167500,176900
3,Arkansas,AR,97200,102300,105100,106300,107300,108700,111400,114700
4,California,CA,479200,458500,421600,383900,366400,371400,385500,409300


In [29]:
income_df.describe()

,state,abbr,mhi_09,mhi_10,mhi_11,mhi_12,mhi_13,mhi_14,mhi_15,mhi_16
count,52,52,52,52,52,52,52,52,52,52
unique,52,52,52,52,52,52,52,52,52,52
top,Kentucky,NH,47465,63277,55975,51273,44874,47764,74551,44758
freq,1,1,1,1,1,1,1,1,1,1


In [30]:
# Use `dropna` to drop any rows where there is missing data
# Notice that index has not been reset and contains a gap (0,2,5,6,7)
# df = df.dropna(axis=0)
# df.head()

In [31]:
# Drop the 'gender' column and reset the index
# Note that after resetting the index, the gap is gone
# df = df.drop(['gender'], axis=1).reset_index(drop=True)
# df.head()

In [8]:
# # Save the cleaned data to a file called `customers_cleaned.csv`
# new_csv = "customers_cleaned.csv"
# df.to_csv(new_csv, index=False)

# Database Creation

In [32]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [33]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///data/flp2Data.sqlite")

In [34]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [36]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Home(Base):
    __tablename__ = 'homeprice'

    id = Column(Integer, primary_key=True)
    state = Column(Text)
    abbr = Column(Text)
    mhp_09 = Column(Integer)
    mhp_10 = Column(Integer)
    mhp_11 = Column(Integer)
    mhp_12 = Column(Integer)
    mhp_13 = Column(Integer)
    mhp_14 = Column(Integer)
    mhp_15 = Column(Integer)
    mhp_16 = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr    

In [37]:
# Use `create_all` to create the median home price table in the database
Base.metadata.create_all(engine)

In [38]:
# Load the cleaned csv file into a pandas dataframe
# new_df = pd.read_csv(new_csv)

In [39]:
# Start with median home price
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
home_data = home_df.to_dict(orient='records')

In [40]:
# Data is just a list of dictionaries that represent each row of data
print(home_data[:5])

[{'state': 'Alabama', 'abbr': 'AL', 'mhp_09': '111900', 'mhp_10': '117600', 'mhp_11': '120800', 'mhp_12': '122300', 'mhp_13': '122500', 'mhp_14': '123800', 'mhp_15': '125500', 'mhp_16': '128500'}, {'state': 'Alaska', 'abbr': 'AK', 'mhp_09': '221300', 'mhp_10': '229100', 'mhp_11': '235100', 'mhp_12': '237900', 'mhp_13': '241800', 'mhp_14': '246300', 'mhp_15': '250000', 'mhp_16': '257100'}, {'state': 'Arizona', 'abbr': 'AZ', 'mhp_09': '218400', 'mhp_10': '215000', 'mhp_11': '197400', 'mhp_12': '175900', 'mhp_13': '165100', 'mhp_14': '162900', 'mhp_15': '167500', 'mhp_16': '176900'}, {'state': 'Arkansas', 'abbr': 'AR', 'mhp_09': '97200', 'mhp_10': '102300', 'mhp_11': '105100', 'mhp_12': '106300', 'mhp_13': '107300', 'mhp_14': '108700', 'mhp_15': '111400', 'mhp_16': '114700'}, {'state': 'California', 'abbr': 'CA', 'mhp_09': '479200', 'mhp_10': '458500', 'mhp_11': '421600', 'mhp_12': '383900', 'mhp_13': '366400', 'mhp_14': '371400', 'mhp_15': '385500', 'mhp_16': '409300'}]


In [42]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [43]:
# Save the reference to the `homeprice table as a variable called `table`
table = sqlalchemy.Table('homeprice', metadata, autoload=True)

In [44]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [45]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), home_data)

In [46]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from homeprice limit 5").fetchall()

[(1, 'Alabama', 'AL', 111900, 117600, 120800, 122300, 122500, 123800, 125500, 128500),
 (2, 'Alaska', 'AK', 221300, 229100, 235100, 237900, 241800, 246300, 250000, 257100),
 (3, 'Arizona', 'AZ', 218400, 215000, 197400, 175900, 165100, 162900, 167500, 176900),
 (4, 'Arkansas', 'AR', 97200, 102300, 105100, 106300, 107300, 108700, 111400, 114700),
 (5, 'California', 'CA', 479200, 458500, 421600, 383900, 366400, 371400, 385500, 409300)]

In [47]:
# Now lets create the median income table


In [48]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [49]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Home(Base):
    __tablename__ = 'income'

    id = Column(Integer, primary_key=True)
    state = Column(Text)
    abbr = Column(Text)
    mhi_09 = Column(Integer)
    mhi_10 = Column(Integer)
    mhi_11 = Column(Integer)
    mhi_12 = Column(Integer)
    mhi_13 = Column(Integer)
    mhi_14 = Column(Integer)
    mhi_15 = Column(Integer)
    mhi_16 = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
# More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr   

In [50]:
# Use `create_all` to create the median home income table in the database
Base.metadata.create_all(engine)

In [51]:
# Next we process median home income
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
income_data = income_df.to_dict(orient='records')

In [52]:
# Data is just a list of dictionaries that represent each row of data
print(income_data[:5])

[{'state': 'Alabama', 'abbr': 'AL', 'mhi_09': '41216', 'mhi_10': '42081', 'mhi_11': '42934', 'mhi_12': '43160', 'mhi_13': '43253', 'mhi_14': '43511', 'mhi_15': '43623', 'mhi_16': '44758'}, {'state': 'Alaska', 'abbr': 'AK', 'mhi_09': '64635', 'mhi_10': '66521', 'mhi_11': '69014', 'mhi_12': '69917', 'mhi_13': '70760', 'mhi_14': '71829', 'mhi_15': '72515', 'mhi_16': '74444'}, {'state': 'Arizona', 'abbr': 'AZ', 'mhi_09': '50296', 'mhi_10': '50448', 'mhi_11': '50752', 'mhi_12': '50256', 'mhi_13': '49774', 'mhi_14': '49928', 'mhi_15': '50255', 'mhi_16': '51340'}, {'state': 'Arkansas', 'abbr': 'AR', 'mhi_09': '38542', 'mhi_10': '39267', 'mhi_11': '40149', 'mhi_12': '40531', 'mhi_13': '40768', 'mhi_14': '41264', 'mhi_15': '41371', 'mhi_16': '42336'}, {'state': 'California', 'abbr': 'CA', 'mhi_09': '60392', 'mhi_10': '60883', 'mhi_11': '61632', 'mhi_12': '61400', 'mhi_13': '61094', 'mhi_14': '61489', 'mhi_15': '61818', 'mhi_16': '63783'}]


In [53]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [54]:
# Save the reference to the `homeprice table as a variable called `table`
table = sqlalchemy.Table('income', metadata, autoload=True)

In [55]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [56]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), income_data)

In [57]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from homeprice limit 5").fetchall()

[(1, 'Alabama', 'AL', 111900, 117600, 120800, 122300, 122500, 123800, 125500, 128500),
 (2, 'Alaska', 'AK', 221300, 229100, 235100, 237900, 241800, 246300, 250000, 257100),
 (3, 'Arizona', 'AZ', 218400, 215000, 197400, 175900, 165100, 162900, 167500, 176900),
 (4, 'Arkansas', 'AR', 97200, 102300, 105100, 106300, 107300, 108700, 111400, 114700),
 (5, 'California', 'CA', 479200, 458500, 421600, 383900, 366400, 371400, 385500, 409300)]